<a href="https://colab.research.google.com/github/IkeSalmonson/Projetos/blob/master/CompiladoTwitSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capitura Twit 
  & salva no drive

In [ ]:
#Imports e classes 
import os
from google.colab import drive
import json

import time

import tweepy

drive.mount('/content/drive')


# Creating a StreamListener class to catch the stream from  tweepy.Stream
class MyStreamListener(tweepy.StreamListener):
  tweetsCounter = 0 #conta quantos tweets foram lidos
  tweetsWanted = 2
  tweets = {}
  def on_status(self,status):

    
    self.tweets[self.tweetsCounter] = status._json 
    self.tweetsCounter +=1

    if self.tweetsWanted < 10:
      print(str(self.tweetsCounter) +" " +status)
    else:
      print(str(self.tweetsCounter) +" ")  
    if self.tweetsCounter < self.tweetsWanted :
      return True
    else :
      return False  

#provavelmente deveria estar usando o on_data e não on_status
  def on_error(self,status_code):
    if status_code == 420:
      return False
      #returning false disconects the stream
       # returning non-False reconnects the stream, with backoff. 


#Salvar em formato JSON no drive com timestamp no nome

class SaveToDrive(MyStreamListener):
  

  savePath = 'drive/My Drive/TwitterStream/'
  
  def save (self):
    t = time.localtime()
    current_time = time.strftime("%d_%m_%H_%M", t)
    fileName = current_time+'_tweets.json'
    with open( self.savePath + fileName, 'w') as fileName :
     json.dump(myStreamListener.tweets, fileName, indent = 4)


print('done'+time.strftime("%d %b %Y %H:%M:%S", time.localtime()))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
done14 Dec 2020 13:48:30


In [ ]:
#rodar para preparar autorização da API

# Transformar em método do MyStreamListener

authPath = 'drive/My Drive/TwitterStream/Auth/'
with open (authPath+'auth.json', 'r',) as file:
  auth = json.load(file)

consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

for key in auth['auth']:
  consumer_key = key['consumer_key']
  consumer_secret =key['consumer_secret'] 
  access_token = key ['access_token']
  access_token_secret =key ['access_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

print('done'+time.strftime("%d %b %Y %H:%M:%S", time.localtime()))

done14 Dec 2020 13:51:04


In [ ]:
#Rodar para capiturar os tweets

#Aparentement ao rodar novamente não redefine filtro e conteudo dos objetos

#Definir as listas para filtro. Os filtros são aditivos ( OR ) qualquer umas das 
filterContent= ['mask'] 
filterLocation = []
filterLanguage = ['en'] # linguas de interesse : en (English) pt (Portuguese)  de (German)  fr (French) ja (Japanese)


tweetsWanted = 10

# Creating a Stream
myStreamListener = MyStreamListener()
myStreamListener.tweetsWanted = tweetsWanted

myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(track=filterContent,
                #locations =filterLocation,
                languages = filterLanguage      
                )

#salva a lista tweets no arquivo
saveToDrive = SaveToDrive(myStreamListener)
saveToDrive.save()

myStream.disconnect()
print('done '+time.strftime("%d %b %Y %H:%M:%S", time.localtime()))

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
done 14 Dec 2020 13:51:10


# Pretratamento e Classificação
com NLTK e SKlearn

In [ ]:
#exatamente igual ao video: + donwloads necessarios, regex e time
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
#Apenas os classificadores que vou usar
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC , NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
from google.colab import drive #Se não montar o drive automaticamente na pasta clicar no botão em "files/mountDrive"
import time
import re
print('done'+time.strftime("%d %b %Y %H:%M:%S", time.localtime()))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
done14 Dec 2020 13:51:15


In [ ]:
#load short pickled classifiers and test accuracy 

object_list = ["short_SGDClassifier_classifier", "short_LinearSVC_classifier",
 "short_NuSVC_classifier"
]
object_short_pickle_load = []

for object_2load in object_list:
  path = '/content/drive/My Drive/TwitterStream/Pickle/'+object_2load+'.pickle' 
  print(path)
  load_file = open( path , "rb") #rb = write binary
  object_short_pickle_load.append(pickle.load(load_file))
  load_file.close()

print(object_short_pickle_load) 



/content/drive/My Drive/TwitterStream/Pickle/short_SGDClassifier_classifier.pickle
/content/drive/My Drive/TwitterStream/Pickle/short_LinearSVC_classifier.pickle
/content/drive/My Drive/TwitterStream/Pickle/short_NuSVC_classifier.pickle
[<SklearnClassifier(SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False))>, <SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>, <SklearnClassifier(NuSVC(break_ties=False, cache_size=200, class

In [ ]:
def find_short_features(review):
  words= nltk.word_tokenize(review)
  features = {}
  for w in words:
    features[w] = (w in words)
  return features 

# short_features = find_short_features(short_documents)

testString = [("I liked it","pos"),("It was bad", "neg"),("I loved it","pos"),("It had bad acting and terrible photography", "neg")]
short_features_set = [(find_short_features(rev),category) for (rev, category) in testString ]

print(short_features_set) 

[({'I': True, 'liked': True, 'it': True}, 'pos'), ({'It': True, 'was': True, 'bad': True}, 'neg'), ({'I': True, 'loved': True, 'it': True}, 'pos'), ({'It': True, 'had': True, 'bad': True, 'acting': True, 'and': True, 'terrible': True, 'photography': True}, 'neg')]


In [ ]:
linearSVC = object_short_pickle_load[1]

for i in range(0,4):
  accuracy_linearSVC_classifier = nltk.classify.accuracy(linearSVC, short_features_set[i:i+1])
  print(accuracy_linearSVC_classifier, i,  short_features_set[i:i+1])


0.0 0 [({'I': True, 'liked': True, 'it': True}, 'pos')]
1.0 1 [({'It': True, 'was': True, 'bad': True}, 'neg')]
1.0 2 [({'I': True, 'loved': True, 'it': True}, 'pos')]
1.0 3 [({'It': True, 'had': True, 'bad': True, 'acting': True, 'and': True, 'terrible': True, 'photography': True}, 'neg')]


# Limpar o twit text

In [ ]:
print(myStreamListener.tweets[0]["text"])
#separar o primeiro : do resto do twit

filter_0 = re.search(r':', myStreamListener.tweets[0]["text"])
#print("Filter_0", filter_0)

textFiltered_1 =myStreamListener.tweets[0]["text"][filter_0.span()[1]:]

print(textFiltered_1)
# Retirar os caracteres indesejados do twit


filtered = re.search(':', myStreamListener.tweets[0]["text"])
print(filtered)

myStreamListener.tweets[0]["text"]

RT @joshtpm: Entire AZ Legislature Shuts Down For A Week After Mask-Free Giuliani Contracts COVID-19 https://t.co/WdyW2d8W3m via @TPM
 Entire AZ Legislature Shuts Down For A Week After Mask-Free Giuliani Contracts COVID-19 https://t.co/WdyW2d8W3m via @TPM
<_sre.SRE_Match object; span=(11, 12), match=':'>


'RT @joshtpm: Entire AZ Legislature Shuts Down For A Week After Mask-Free Giuliani Contracts COVID-19 https://t.co/WdyW2d8W3m via @TPM'

In [ ]:
print(myStreamListener.tweets[0]["text"][filtered.span()[1]:])

 201207 MX Talk Tok

[MONSTAX_KH]
Monbebe, everyone have to wear mask well!!!!
We really need to protect our bodies!!
